### Import neccessary lib.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

### read ticker csv and import API token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

### Test the API link

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# print(data.status_code)
data


{'companyName': 'Apple Inc',
 'marketcap': 2526112713681,
 'week52high': 157.83,
 'week52low': 114,
 'week52highSplitAdjustOnly': 159.17,
 'week52lowSplitAdjustOnly': 115.65,
 'week52change': 0.3102414825059691,
 'sharesOutstanding': 16666522887,
 'float': 0,
 'avg10Volume': 65151716,
 'avg30Volume': 70805775,
 'day200MovingAvg': 142.12,
 'day50MovingAvg': 147.89,
 'employees': 148494,
 'ttmEPS': 11.45,
 'ttmDividendRate': 0.8845106738452237,
 'dividendYield': 0.005855814322409261,
 'nextDividendDate': '',
 'exDividendDate': '2021-11-01',
 'nextEarningsDate': '2022-01-25',
 'peRatio': 13.206515849176784,
 'beta': 1.4146106085891832,
 'maxChangePercent': 57.46717818692503,
 'year5ChangePercent': 4.883824221829771,
 'year2ChangePercent': 1.40596567924913,
 'year1ChangePercent': 0.3077841214138658,
 'ytdChangePercent': 0.1452707373564721,
 'month6ChangePercent': 0.163288744907799,
 'month3ChangePercent': 0.03519055156048466,
 'month1ChangePercent': 0.05944324427659867,
 'day30ChangePercen

In [4]:
data['year5ChangePercent']

4.883824221829771

### batch append df from API link

In [5]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
        
symbol_groups = list(chunks(stocks['Ticker'],100))
# print(symbol_groups)
symbol_strings = []
for i in range(0,len(symbol_groups)):
#         print(symbol_groups[i])
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
for symbol_string in symbol_strings:
    print(symbol_string)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [6]:
my_columns = ['Ticker','Stock Price', '5 Years Price Return','Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,5 Years Price Return,Number of shares to buy


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
#         print(symbol)
        final_dataframe = final_dataframe.append(pd.Series(
            [symbol,
             data[symbol]['price'],
             data[symbol]['stats']['year5ChangePercent'], 
             'N/A'],
            index = my_columns),
            ignore_index = True)
final_dataframe
# x= final_dataframe['AAPL']['5 Years Price Return']
# print(x)

,Ticker,Stock Price,5 Years Price Return,Number of shares to buy
0,A,162.20,2.656704,N/A
1,AAL,21.74,-0.471571,N/A
2,AAP,241.56,0.719452,N/A
3,AAPL,149.89,4.98803,N/A
4,ABBV,118.51,1.349898,N/A
...,...,...,...,...
500,YUM,128.80,1.246175,N/A
501,ZBH,135.54,0.375233,N/A
502,ZBRA,609.20,7.955517,N/A
503,ZION,67.53,0.387225,N/A


### get single Ticker data

In [9]:
specific_stock = final_dataframe.loc[final_dataframe['Ticker'] =='AAPL']
specific_stock 

,Ticker,Stock Price,5 Years Price Return,Number of shares to buy
3,AAPL,149.89,4.98803,N/A


### 5 years price return wise TOP 50 Stock

In [10]:
final_dataframe.sort_values('5 Years Price Return',ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Stock Price,5 Years Price Return,Number of shares to buy
0,31,AMD,144.30,21.093144,N/A
1,345,NVDA,306.62,17.19846,N/A
2,358,PAYC,496.50,11.170318,N/A
3,195,FTNT,347.78,10.145443,N/A
4,150,DXCM,632.63,9.391689,N/A
5,502,ZBRA,609.20,7.955517,N/A
6,319,MSCI,662.00,7.566988,N/A
7,339,NOW,673.33,7.220578,N/A
8,24,ALGN,710.69,7.081328,N/A
9,85,CDNS,181.05,6.337549,N/A


### get input portfolio size

In [11]:
portfolio_size =input('enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Thats not a number! \nPlease try again:')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

enter the value of your portfolio:1000000


### create number of share column

In [12]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

C:\Users\lenovo\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Stock Price,5 Years Price Return,Number of shares to buy
0,31,AMD,144.30,21.093144,138
1,345,NVDA,306.62,17.19846,65
2,358,PAYC,496.50,11.170318,40
3,195,FTNT,347.78,10.145443,57
4,150,DXCM,632.63,9.391689,31
5,502,ZBRA,609.20,7.955517,32
6,319,MSCI,662.00,7.566988,30
7,339,NOW,673.33,7.220578,29
8,24,ALGN,710.69,7.081328,28
9,85,CDNS,181.05,6.337549,110


In [13]:
# final_dataframe.to_csv('top_50_stocks_for_buy.csv')